In [1]:
from loader import getLoader, path_to_train, path_to_test,loader_batch_size_train,loader_batch_size_test, path_to_slack,loader_batch_size_slack

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.nn.utils.rnn import pad_sequence
import pandas as pd


device = torch.device('cpu')

In [2]:
print("Paths:",path_to_train,path_to_test,path_to_slack)
print("batch size train:",loader_batch_size_train)
print("batch size test:",loader_batch_size_test)
print("batch size slack:",loader_batch_size_slack)
skip_training = True


Paths: data/large/processed_train.csv data/large/processed_test.csv data/large/slack_test.csv
batch size train: 512
batch size test: 5
batch size slack: 1


In [3]:
class Classifier(nn.Module):
    def __init__(self, dictionary_size, hidden_size, output_size=2):
        super(Classifier, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(dictionary_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)
        self.linear = nn.Linear(hidden_size,output_size)

    def forward(self, pad_seqs, seq_lengths, hidden):
        """
        Args:
          pad_seqs: Tensor [max_seq_length, batch_size, 1]
          seq_lengths: list of sequence lengths
          hidden: Tensor [1, batch_size, hidden_size]

        Returns:
          outputs: Tensor [max_seq_length, batch_size, hidden_size]
          hidden: Tensor [1, batch_size, hidden_size]
        """
        # YOUR CODE HERE
        batch_size = pad_seqs.shape[1]
        
        embedded = self.embedding(pad_seqs).view(pad_seqs.shape[0], pad_seqs.shape[1], -1)

        packed = pack_padded_sequence(embedded, seq_lengths, batch_first = False)
        
        self.lstm.flatten_parameters()
        _,hidden = self.lstm(packed)
        
        fc = self.linear(hidden[0])

        return fc

    def init_hidden(self, batch_size=1, device=device):
        return torch.zeros(1, batch_size, self.hidden_size, device=device)

In [4]:
hidden_size = 128
dictionary_size = 20000
classifier = Classifier(dictionary_size, hidden_size).to(device)

In [5]:
trainloader = getLoader(train=True,mini=False)

In [6]:
#Quick output test
for i, batch in enumerate(trainloader):
    print("iter", i)
    pad_input_seqs, input_seq_lengths, pad_target_seqs, target_seq_lengths = batch
    batch_size = pad_input_seqs.size(1)
    pad_input_seqs, pad_target_seqs = pad_input_seqs.to(device), pad_target_seqs.to(device)

    classifier_hidden = classifier.init_hidden(batch_size, device)
    print("in:",pad_input_seqs.size())
    classifier_hidden = classifier(pad_input_seqs, input_seq_lengths, classifier_hidden)
    print(classifier_hidden)
    print("out:",classifier_hidden.size())
    break

iter 0
in: torch.Size([37, 512, 1])
tensor([[[ 0.0857,  0.1926],
         [-0.0230,  0.1797],
         [ 0.0260,  0.0842],
         ...,
         [ 0.0713,  0.1668],
         [ 0.0713,  0.1668],
         [ 0.0513,  0.0523]]], grad_fn=<AddBackward0>)
out: torch.Size([1, 512, 2])


In [7]:
n_epochs = 10

classifier_optimizer = optim.Adam(classifier.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [8]:
for epoch in range(n_epochs):
    running_loss = 0.0
    print_every = 200 
    print("Epoch", epoch+1)
    
    for i, batch in enumerate(trainloader):
        classifier_optimizer.zero_grad()
        
        pad_input_seqs, input_seq_lengths, pad_target_seqs, target_seq_lengths = batch
        batch_size = pad_input_seqs.size(1)
        pad_input_seqs, pad_target_seqs = pad_input_seqs.to(device), pad_target_seqs.to(device)

        classifier_hidden = classifier.init_hidden(batch_size, device)

        # Encode input sequence
        classifier_hidden = classifier(pad_input_seqs, input_seq_lengths, classifier_hidden)

        #print(classifier_hidden.size(),pad_target_seqs.size())
        loss = criterion(classifier_hidden.view(batch_size,2), pad_target_seqs)
        loss.backward()
        

        classifier_optimizer.step()
        running_loss += loss.item()
        if (i % print_every) == (print_every-1) or i == (len(trainloader) // trainloader.batch_size):
            print('[%d, %5d] loss: %.4f' % (epoch+1, i+1, running_loss/print_every))
            running_loss = 0.0

        if skip_training:
            break
    if skip_training:
        break

print('Finished Training')

Epoch 1
Finished Training


In [5]:
# Save the model to disk, submit these files together with your notebook
classifier_filename = 'classifier_model.pth'
if not skip_training:
    try:
        torch.save(classifier.state_dict(), classifier_filename)
        print('Model saved to %s' % (classifier_filename))

    except:
        pass
else:
    classifier = Classifier(dictionary_size, hidden_size)
    classifier.load_state_dict(torch.load(classifier_filename, map_location=lambda storage, loc: storage))
    print('Classifier loaded from %s.' % classifier_filename)
    classifier = classifier.to(device)
    classifier.eval()

Classifier loaded from classifier_model.pth.


In [10]:
testloader = getLoader(train=False,mini=False)


In [11]:
print_every_test = 20000
def compute_accuracy(classifier, testloader):
    classifier.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for pad_input_seqs, input_seq_lengths, targets, _ in testloader:
            batch_size = pad_input_seqs.size(1)
            
            pad_input_seqs, targets = pad_input_seqs.to(device), targets.to(device)
            
            init_hidden = classifier.init_hidden(batch_size, device)
            output = classifier(pad_input_seqs, input_seq_lengths, init_hidden)
            
            out_flat = output.detach().numpy().argmax(axis=2)
            predicted = torch.tensor(out_flat)
            #print("pred", predicted)
            #print("true", targets)
            
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
            #print("tot",total)
            #print("cor",correct)
            if (total % print_every_test == 0):
                print("Counted:",total,"accuracy",correct / total)
    return correct / total

In [12]:
print("Final accuracy:",compute_accuracy(classifier,testloader))

Counted: 20000 accuracy 0.831
Counted: 40000 accuracy 0.82795
Counted: 60000 accuracy 0.8256
Counted: 80000 accuracy 0.82655
Counted: 100000 accuracy 0.82632
Counted: 120000 accuracy 0.8268416666666667
Counted: 140000 accuracy 0.8270357142857143
Counted: 160000 accuracy 0.8275625
Counted: 180000 accuracy 0.8276388888888889
Counted: 200000 accuracy 0.827915
Counted: 220000 accuracy 0.8279909090909091
Counted: 240000 accuracy 0.827875
Counted: 260000 accuracy 0.8280961538461539
Counted: 280000 accuracy 0.8280571428571428
Counted: 300000 accuracy 0.8277633333333333
Counted: 320000 accuracy 0.827578125
0.827578125


In [6]:
slackloader = getLoader(train=False,mini=False, slack=True)

In [7]:
word_df = pd.read_csv("data/words.csv",squeeze=True)
index_word = {x:y for x,y in enumerate(word_df["0"])}
word_index = {y:x for x,y in enumerate(word_df["0"])}

In [32]:
print_every_slack = 20
def evaluate_slack(classifier, testloader):
    classifier.eval()
    negative = 0
    positive = 0
    total = 0
    max_pos = 0
    max_pos_idx = None
    max_neg = 0
    max_neg_idx = None
    
    with torch.no_grad():
        for i, (pad_input_seqs, input_seq_lengths, targets , _) in enumerate(testloader):
            batch_size = pad_input_seqs.size(1)
            
            pad_input_seqs = pad_input_seqs.to(device)
            
            init_hidden = classifier.init_hidden(batch_size, device)
            output = classifier(pad_input_seqs, input_seq_lengths, init_hidden)
            
            out_flat = output.detach().numpy().argmax(axis=2)
            predicted = torch.tensor(out_flat)
            
            #print(i, "both",output.numpy().flatten())
            
            #print("ind pos",output.numpy().flatten()[1])
            #print("ind neg",output.numpy().flatten()[0])
            
            if (max_pos < output.numpy().flatten()[1]):
                max_pos = output.numpy().flatten()[1]
                max_pos_idx = i
            
            if (max_neg < output.numpy().flatten()[0]):
                max_neg = output.numpy().flatten()[0]
                max_neg_idx = i

            #print("sent", pad_input_seqs)
            #print(output)
            #print("pred", predicted)
            #print("true", targets)
            
            total += targets.size(0)
            positive += (predicted == 1).sum().item()
            negative += (predicted == 0).sum().item()
            #print("pos",positive)
            #print("neg",negative)

            if (total % print_every_slack == 0):
                print("Counted:",total,"positive",positive / total,"negative",negative / total)
    return positive, negative, total, max_pos_idx, max_neg_idx

In [33]:
evaluate_slack(classifier,slackloader)

Counted: 20 positive 0.6 negative 0.4
Counted: 40 positive 0.65 negative 0.35
Counted: 60 positive 0.5833333333333334 negative 0.4166666666666667
Counted: 80 positive 0.5625 negative 0.4375
Counted: 100 positive 0.56 negative 0.44
Counted: 120 positive 0.5583333333333333 negative 0.44166666666666665
Counted: 140 positive 0.55 negative 0.45
Counted: 160 positive 0.55 negative 0.45
Counted: 180 positive 0.5444444444444444 negative 0.45555555555555555
Counted: 200 positive 0.555 negative 0.445
Counted: 220 positive 0.5272727272727272 negative 0.4727272727272727
Counted: 240 positive 0.5291666666666667 negative 0.4708333333333333
Counted: 260 positive 0.5269230769230769 negative 0.47307692307692306
Counted: 280 positive 0.5321428571428571 negative 0.46785714285714286
Counted: 300 positive 0.5333333333333333 negative 0.4666666666666667
Counted: 320 positive 0.540625 negative 0.459375
Counted: 340 positive 0.5411764705882353 negative 0.4588235294117647
Counted: 360 positive 0.544444444444444

(703, 706, 1409, 1022, 131)

In [37]:
for i,sent in enumerate(slackloader):
    if (i == 1022):
        for word in sent[0].numpy().flatten():
            if (word != 2 and word != 3 and word != 0):
                print(index_word[word],end=" ")

set the channel purpose for finding project partners 

In [38]:
for i,sent in enumerate(slackloader):
    if (i == 131):
        for word in sent[0].numpy().flatten():
            if (word != 2 and word != 3 and word != 0):
                print(index_word[word],end=" ")

are there any tutorial sessions this week ? the booking option is not available on CTX edited 